In [1]:
%defaultDatasource jdbc:h2:file:~/data/caminhos/caminhos

# Estradas e Trajetos

Considere o diagrama abaixo representa graficamente um modelo relacional de tabelas que controlam cidades, estradas e trajetos entre cidades. A seguir a descrição o papel de cada tabela:
* _Cidades_: mantém um cadastro de cidades.
* _Estradas_: registra estradas que ligam uma cidade (cidade_origem) a outra (cidade_destino), bem como sua quilometragem.
* _Trajeto_: cada registro da tabela Trajeto identifica um trajeto, que consiste em uma sequência ordenada de estradas que ligam duas cidades (cidade_origem e cidade_destino), por exemplo, um trajeto entre Salvador e Curitiba, pode envolver uma sequência de estradas: Salvador-Belo Horizonte, Belo Horizonte-São Paulo e São Paulo-Curitiba. Um Trajeto agrega um conjunto de Segmentos.
* _Segmento_: associa estradas a trajetos. O campo ordem é um campo numérico sequencial (iniciado de 1 para cada trajeto) usado para ordenar os segmentos (estradas) dentro de um trajeto.

![UML](uml-estradas.png) ![Relacional](relacional-estradas.png)

## Exercício 1

Escreva uma sentença em SQL que calcule a quilometragem total de cada Trajeto.

In [2]:
CREATE TABLE Cidade (
  cidadeid VARCHAR(5) NOT NULL ,
  nome VARCHAR(100) ,
  PRIMARY KEY (cidadeid) );

INSERT INTO Cidade VALUES ('SSA', 'Salvador');
INSERT INTO Cidade VALUES ('SP',  'Sao Paulo');
INSERT INTO Cidade VALUES ('RJ',  'Rio de Janeiro');
INSERT INTO Cidade VALUES ('FLO', 'Florianopolis');
INSERT INTO Cidade VALUES ('CUR', 'Curitiba');

CREATE TABLE Estrada (
  estradaid VARCHAR(5) NOT NULL ,
  nome VARCHAR(100) ,
  cidade_origem VARCHAR(5) NOT NULL ,
  cidade_destino VARCHAR(5) NOT NULL ,
  extensao_km INT ,
  PRIMARY KEY (estradaid) ,
  FOREIGN KEY (cidade_origem )
    REFERENCES Cidade (cidadeid )
    ON DELETE NO ACTION
    ON UPDATE NO ACTION,
  FOREIGN KEY (cidade_destino )
    REFERENCES Cidade (cidadeid )
    ON DELETE NO ACTION
    ON UPDATE NO ACTION);

INSERT INTO Estrada VALUES ('4545', 'Estrada 4545', 'SSA', 'RJ', 1800);
INSERT INTO Estrada VALUES ('2835', 'Estrada 2835', 'RJ', 'SP', 400);
INSERT INTO Estrada VALUES ('1012', 'Estrada 1012', 'SP', 'FLO', 600);
INSERT INTO Estrada VALUES ('1212', 'Estrada 1212', 'SP', 'CUR', 800);
INSERT INTO Estrada VALUES ('6032', 'Estrada 6032', 'FLO', 'CUR', 200);

CREATE TABLE Trajeto (
  trajetoid VARCHAR(5) NOT NULL ,
  cidade_origem VARCHAR(5) NOT NULL ,
  cidade_destino VARCHAR(5) NOT NULL ,
  PRIMARY KEY (trajetoid) ,
  FOREIGN KEY (cidade_origem )
    REFERENCES Cidade (cidadeid )
    ON DELETE NO ACTION
    ON UPDATE NO ACTION,
  FOREIGN KEY (cidade_destino )
    REFERENCES Cidade (cidadeid )
    ON DELETE NO ACTION
    ON UPDATE NO ACTION);

INSERT INTO Trajeto VALUES ('TJ001', 'SSA', 'SP');
INSERT INTO Trajeto VALUES ('TJ002', 'SSA', 'FLO');
INSERT INTO Trajeto VALUES ('TJ003', 'SP', 'CUR');
INSERT INTO Trajeto VALUES ('TJ004', 'SP', 'CUR');
INSERT INTO Trajeto VALUES ('TJ005', 'RJ', 'CUR');

CREATE TABLE Segmento (
  trajetoid VARCHAR(5) NOT NULL ,
  estradaid VARCHAR(5) NOT NULL ,
  ordem INT ,
  PRIMARY KEY (trajetoid, estradaid) ,
  FOREIGN KEY (estradaid )
    REFERENCES Estrada (estradaid )
    ON DELETE NO ACTION
    ON UPDATE NO ACTION,
  FOREIGN KEY (trajetoid )
    REFERENCES Trajeto (trajetoid )
    ON DELETE NO ACTION
    ON UPDATE NO ACTION);

INSERT INTO Segmento VALUES ('TJ001', '4545', 1);
INSERT INTO Segmento VALUES ('TJ001', '2835', 2);
INSERT INTO Segmento VALUES ('TJ002', '4545', 1);
INSERT INTO Segmento VALUES ('TJ002', '2835', 2);
INSERT INTO Segmento VALUES ('TJ002', '1012', 3);
INSERT INTO Segmento VALUES ('TJ003', '4545', 1);
INSERT INTO Segmento VALUES ('TJ003', '2835', 2);
INSERT INTO Segmento VALUES ('TJ003', '1012', 3);
INSERT INTO Segmento VALUES ('TJ004', '4545', 1);
INSERT INTO Segmento VALUES ('TJ004', '2835', 2);
INSERT INTO Segmento VALUES ('TJ004', '1012', 3);
INSERT INTO Segmento VALUES ('TJ004', '6032', 4);
INSERT INTO Segmento VALUES ('TJ005', '1012', 1);
INSERT INTO Segmento VALUES ('TJ005', '6032', 2);

SELECT * FROM Cidade;
SELECT * FROM Estrada;
SELECT * FROM Trajeto;
SELECT * FROM Segmento;

# Cronograma de Tarefas

Considere uma tabela contendo tarefas (Tasks) a ser executadas e a dependência entre elas (Dependency). Na dependência entre a Tarefa A e B é computado o tempo para que a Tarefa A se conclua e se inicie a Tarefa B.

In [3]:
DROP TABLE IF EXISTS Task;
DROP TABLE IF EXISTS Dependency;

CREATE TABLE Task (
  id VARCHAR(30) NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(id)
) AS SELECT id, name
     FROM CSVREAD('../../data/schedule/task.csv');

SELECT * FROM Task;

CREATE TABLE Dependency (
  before VARCHAR(30) NOT NULL,
  after VARCHAR(30) NOT NULL,
  time INT NOT NULL,
  PRIMARY KEY(before, after),
  FOREIGN KEY(before)
    REFERENCES Task(id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(after)
    REFERENCES Task(id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT before, after, time
     FROM CSVREAD('../../data/schedule/dependency.csv');

SELECT * FROM Dependency;

# Versão dos caminhos no Neo4J / Cypher

Entre na sandbox do Cypher (https://neo4j.com/sandbox-v2/?ref=hcard#) e execute os comandos.

## Criando um grafo de tarefas no Cypher
```
LOAD CSV WITH HEADERS FROM 'https://github.com/santanche/lab2learn/raw/master/data/schedule/task.csv' AS line
CREATE (:Task {id: line.id, name: line.name})

LOAD CSV WITH HEADERS FROM 'https://github.com/santanche/lab2learn/raw/master/data/schedule/dependency.csv' AS line
MATCH (before:Task {id:line.before})
MATCH (after:Task {id:line.after})
CREATE (before)-[:Dependency {time:toInteger(line.time)}]->(after)

MATCH (t:Task)
RETURN t
```

## Caminho entre a tarefa Preparação e Fatura
O * indica que é aceito um número variável de arestas entre os dois.
```
MATCH path=(:Task {id: "Preparação"})-[:Dependency*]->(:Task {id: "Fatura"})
RETURN path
```

## Tamanho do caminho
A função length() calcula o número de arestas.
```
MATCH path=(:Task {id: "Preparação"})-[:Dependency*]->(:Task {id: "Fatura"})
RETURN length(path)
```

# Somando o tempo do caminho
A cláusula UNWIND transforma o caminho em um conjunto de valores e a função sum() os soma.
```
MATCH path=(:Task {id: "Preparação"})-[d:Dependency*]->(:Task {id: "Fatura"})
UNWIND d as p
RETURN sum(p.time)
```

## Exercício 2
Reproduza estas operação em SQL com Java (veja código Java de base no notebook associado.